In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import recall_score

In [2]:
heart=pd.read_csv('framingham.csv')
heart.dropna(inplace=True)
X=heart.drop('TenYearCHD', axis=1)
y=heart['TenYearCHD']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=.2,stratify=y)
#################
# Random Forest #
#################
rf_imp=[]
for i in tqdm(range(0,100)):
    X_train_1, X_val, y_train_1, y_val=train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    rf_md=RandomForestClassifier(n_estimators=500, 
                                 max_depth=5).fit(X_train_1, y_train_1)
    rf_imp.append(rf_md.feature_importances_)
rf_imp=pd.DataFrame(rf_imp, columns=X.columns.tolist())
rf_imp=rf_imp.apply(np.mean,axis=0).sort_values(ascending=False)
rf_imp

  0%|          | 0/100 [00:00<?, ?it/s]

sysBP              0.186252
age                0.184066
diaBP              0.120829
glucose            0.102826
BMI                0.082528
totChol            0.081342
prevalentHyp       0.057079
heartRate          0.047583
cigsPerDay         0.043041
male               0.029710
education          0.024558
BPMeds             0.012338
diabetes           0.011318
currentSmoker      0.008803
prevalentStroke    0.007728
dtype: float64

In [5]:
to_select=['sysBP', 'age', 'diaBP', 'glucose', 'BMI']
rf_md=rf_md=RandomForestClassifier(n_estimators=500, 
                                 max_depth=5).fit(X_train[to_select], y_train)
rf_pred=rf_md.predict_proba(X_test[to_select])[:,1]
rf_label=np.where(rf_pred<0.1, 0, 1)
print(f"The recall score of the rf model is {recall_score(y_test, rf_label)}")

The recall score of the rf model is 0.8571428571428571


In [8]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [9]:
#################
# Extra Trees #
#################
et_imp=[]
for i in tqdm(range(0,100)):
    X_train_1, X_val, y_train_1, y_val=train_test_split(X_train,y_train,test_size=.2,stratify=y_train)
    et_md=ExtraTreesClassifier(n_estimators=500, 
                                 max_depth=5).fit(X_train_1, y_train_1)
    et_imp.append(rf_md.feature_importances_)
et_imp=pd.DataFrame(et_imp, columns=X.columns.tolist())
et_imp=et_imp.apply(np.mean,axis=0).sort_values(ascending=False)
et_imp

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [10]:
to_select=['prevalentHyp','age','sysBP','glucose']
et_md=et_md=ExtraTreesClassifier(n_estimators=500, 
                                 max_depth=5).fit(X_train[to_select], y_train)
et_pred=et_md.predict_proba(X_test[to_select])[:,1]
et_label=np.where(et_pred<0.1, 0, 1)
print(f"The recall score of the rf model is {recall_score(y_test, et_label)}")

The recall score of the rf model is 0.8660714285714286


In [11]:
# From the above results, I would use Extra Trees to predict TenYearCHD.